In [13]:
# Task 7: USA Drilldown Analysis
# Using usa_county_wise.csv:
# 1. Aggregate county data to state level.
# 2. Identify top 10 affected states.
# 3. Detect data skew across states.
# 4. Explain skew impact in distributed systems.

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [29]:
spark = SparkSession.builder.appName("USA Drilldown Analysis").master("yarn").getOrCreate()
spark

26/02/22 11:50:44 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [30]:
usa_county_wise = spark.read.parquet("hdfs:///data/covid/staging/usa_county_wise")
usa_county_wise.show()

[Stage 1:>                                                          (0 + 1) / 1]

+--------+----+----+-----+-------+----------+--------------+--------------+------------------+------------------+---------+------+
|     UID|iso2|iso3|code3|   FIPS|    Admin2|Province_State|Country_Region|               Lat|             Long_|Confirmed|Deaths|
+--------+----+----+-----+-------+----------+--------------+--------------+------------------+------------------+---------+------+
|84013149|  US| USA|  840|13149.0|     Heard|       Georgia|            US|       33.29686567|      -85.12886832|       11|     1|
|84013151|  US| USA|  840|13151.0|     Henry|       Georgia|            US|        33.4533428|-84.14814659999998|      492|    13|
|84013153|  US| USA|  840|13153.0|   Houston|       Georgia|            US|       32.45802497|      -83.66879087|      234|    14|
|84013155|  US| USA|  840|13155.0|     Irwin|       Georgia|            US|       31.60213036|      -83.27501387|       15|     1|
|84013157|  US| USA|  840|13157.0|   Jackson|       Georgia|            US|       3

### 1. Aggregate county data to state level.

In [31]:
state_level_aggregation = usa_county_wise.groupBy("Province_State")\
    .agg(avg("Lat").alias("Avg_Lat"),\
         avg("Long_").alias("Avg_Long_"),\
         count_distinct("Admin2").alias("No_of_Admins"),\
         sum("Confirmed").alias("Total Confirmed"),\
         sum("Deaths").alias("Total Deaths"),
        )

In [32]:
state_level_aggregation.show()

[Stage 2:=============================>                             (1 + 1) / 2]

+--------------------+-------------------+-------------------+------------+---------------+------------+
|      Province_State|            Avg_Lat|          Avg_Long_|No_of_Admins|Total Confirmed|Total Deaths|
+--------------------+-------------------+-------------------+------------+---------------+------------+
|                Utah| 37.528405034594606|-105.52756603783786|          37|        1596769|       14229|
|              Hawaii| 15.020202147142856|-112.37374034285718|           7|          92930|        2003|
|           Minnesota|  44.52837223842696| -92.30890913865169|          89|        2637428|      107860|
|                Ohio|  39.41487715044446| -80.99462116200002|          90|        4275663|      222951|
|Northern Mariana ...| 15.097899999999987|  145.6738999999998|           1|           2741|         230|
|              Oregon|  42.15069537605262| -115.0833975131579|          38|         695008|       18165|
|            Arkansas|  34.00508706844154| -90.03309571

In [33]:
state_level_aggregation.write.mode("overwrite").parquet("hdfs:///data/covid/analytics/state_level_aggregation")

### 2. Identify top 10 affected states.

In [34]:
top10_affected_states = state_level_aggregation.select("Province_State","Total Confirmed").orderBy(col("Total Confirmed").desc())

In [35]:
top10_affected_states.limit(10).show()

+--------------+---------------+
|Province_State|Total Confirmed|
+--------------+---------------+
|      New York|       39808447|
|    California|       17618695|
|    New Jersey|       16506714|
|         Texas|       12698726|
|       Florida|       12657802|
|      Illinois|       11900637|
| Massachusetts|        9874030|
|  Pennsylvania|        8096993|
|       Georgia|        6859759|
|      Michigan|        6690544|
+--------------+---------------+



In [36]:
top10_affected_states.write.mode("overwrite").parquet("hdfs:///data/covid/analytics/top10_affected_states")

###  3. Detect data skew across states.

In [37]:
state_distribution = (
    usa_county_wise.groupBy("Province_State")
      .agg(count("*").alias("record_count"))
      .orderBy(col("record_count").desc())
)

state_distribution.show()

+--------------+------------+
|Province_State|record_count|
+--------------+------------+
|         Texas|       48128|
|       Georgia|       30268|
|      Virginia|       25380|
|      Kentucky|       22936|
|      Missouri|       22184|
|        Kansas|       20116|
|      Illinois|       19552|
|North Carolina|       19176|
|          Iowa|       18988|
|     Tennessee|       18236|
|      Nebraska|       17860|
|       Indiana|       17672|
|          Ohio|       16920|
|     Minnesota|       16732|
|      Michigan|       16356|
|   Mississippi|       15792|
|   Puerto Rico|       15040|
|      Oklahoma|       14852|
|      Arkansas|       14476|
|     Wisconsin|       13912|
+--------------+------------+
only showing top 20 rows


In [40]:
stats = state_distribution.agg(
    avg("record_count").alias("mean"),
    stddev("record_count").alias("std")
).first()

mean_value = stats["mean"]
std_value = stats["std"]

state_level_skew = state_distribution.withColumn(
    "skew",
    (col("record_count") - mean_value) / std_value
).orderBy(col("skew").desc())

state_level_skew.show()

+--------------+------------+-------------------+
|Province_State|record_count|               skew|
+--------------+------------+-------------------+
|         Texas|       48128|  4.142461036456503|
|       Georgia|       30268|  2.159061635094378|
|      Virginia|       25380| 1.6162365357742177|
|      Kentucky|       22936| 1.3448239861141373|
|      Missouri|       22184| 1.2613124323725742|
|        Kansas|       20116| 1.0316556595832755|
|      Illinois|       19552| 0.9690219942771032|
|North Carolina|       19176| 0.9272662174063216|
|          Iowa|       18988| 0.9063883289709309|
|     Tennessee|       18236| 0.8228767752293678|
|      Nebraska|       17860| 0.7811209983585862|
|       Indiana|       17672| 0.7602431099231954|
|          Ohio|       16920| 0.6767315561816323|
|     Minnesota|       16732| 0.6558536677462415|
|      Michigan|       16356|   0.61409789087546|
|   Mississippi|       15792| 0.5514642255692875|
|   Puerto Rico|       15040|0.46795267182772443|


In [41]:
state_level_skew.write.mode("overwrite").parquet("hdfs:///data/covid/analytics/state_level_skew")

### 4. Explain skew impact in distributed systems.
```
Data skew happens when:
1. Some keys (in this case it is states) have much more data
2. Other keys have very little

Problems Caused by Skew:
* Uneven CPU usage
* Long shuffle stages
* Memory spill
* Straggler tasks
* Increased job completion time
```

In [42]:
spark.stop()